<a href="https://colab.research.google.com/github/VettoriKev/Machine-Learning/blob/main/Vettori_Tp7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Trabajo Practico 7
## Reducción de la dimensionalidad
----


##Encuestas de salud y trabajo
El objetivo de este proyecto es describir las disparidades entre generos en la ultima Encuesta Nacional a trabajadores sobre Condiciones de Empleo, Trabajo, Salud y Seguridad (ECETSS 2018).

Información complementaria [link](https://www.argentina.gob.ar/srt/observatorio-srt/encuestas-salud-trabajo/ECETSS-2018)


In [ ]:
import numpy as np
import pandas as pd

dataroot = "https://raw.githubusercontent.com/manlio99/Materia-de-aprendizaje/master/3_MidtermProjects/trabajadores/ecetss_ocupados.csv"
df = pd.read_csv(dataroot)
df.sample(10)

,codmuestra,hogar,region,ponderaocup,C0P10.2,C0P10.3,C3P16.1,C3P16.2,nivel_ed,C3P16.6,C3P16.6.1cod,C3P16.7,C3P16.8,C1P2.1,C1P2.6,cat_ocup,rama_caes,ocupa_cno,ramacaes_o,sector,establecimiento,sit_oc,tamanio,antigestab_a,antigpuesto,registracion,upreg,tickets,quien_contrato,C1P3.1.10,C1P3.1.11,C1P3.1.12,C1P3.1.15,C1P3.1.16,C1P3.2.10,C1P3.2.11,C1P3.2.12,C1P3.3.1,C1P3.3.2,antigestab_sd,...,C2P12.15,C1P3.3.14,C1P3.3.19,aumentosal,C2P12.11,C1P3.3.18,ingreso_op,C2P13.1.1,C2P13.1.2,C2P13.1.3,C2P13.1.4,C2P13.1.5,C2P13.1.6,C2P13.1.7,C2P13.1.8,C2P13.1.9,C2P13.1.10,C2P13.4.2,C2P13.4.3,C2P13.4.4,C2P13.4.5,C2BP13.2,C2BP13.3,C3P14.2,C3P14.3,C3P14.4,C3P14.5,C3P14.6,C3P14.7,C3P14.8,trabdom_noremu,C3P15.1,C3P15.2,C3P15.3,C3P17.1,edadhogar,pershog_ag,edad_0a5,edad_6a11,edad_12a17
7322,070-0090-0097,1,42,195.512155,1,51,1,2,2,1,NaN,NaN,1,1,1,3,8401,80313,8,1,NaN,1,6.0,30.0,6.0,1.0,4.0,3.0,1.0,1.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,2.0,NaN,48000,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,00:00,3,3,10,2,2,1,1,2
8834,095-0017-0002,1,44,312.096881,1,62,1,2,4,1,NaN,NaN,1,1,2,2,4904,34123,6,2,5.0,7,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,06:00,1,5,5,1,2,1,1,1
4119,026-0028-0333,1,44,2509.112532,1,40,2,2,3,1,NaN,NaN,1,1,1,3,9700,57314,11,2,5.0,4,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.06,...,NaN,2.0,2.0,2.0,NaN,NaN,12000,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,00:30,1,3,9,5,3,1,1,2
7020,070-0004-0095,1,42,783.502811,2,39,2,2,8,1,NaN,NaN,1,1,1,3,8600,40312,10,1,NaN,2,99.0,1.0,1.0,2.0,3.0,99.0,1.0,NaN,NaN,NaN,NaN,NaN,99.0,NaN,99.0,NaN,NaN,NaN,...,8.0,NaN,NaN,2.0,NaN,NaN,0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,00:00,4,3,9,2,3,1,2,2
789,002-0068-0009,1,10,800.273722,1,20,1,2,6,1,NaN,NaN,1,2,1,3,5601,53313,5,2,1.0,1,3.0,2.0,1.0,1.0,4.0,1.0,1.0,1.0,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,1.0,4.0,NaN,9000,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,50.0,1.0,1.0,NaN,NaN,NaN,15000.0,1,04:00,4,5,9,6,2,1,1,1
5599,050-0032-0185,1,42,477.704732,1,55,2,3,8,1,NaN,NaN,1,1,1,2,6900,11131,7,2,1.0,7,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,30000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,00:00,1,5,6,6,1,1,1,1
8227,082-0084-0437,1,43,2297.762444,2,17,2,2,3,1,NaN,NaN,1,1,1,3,9700,57314,11,2,5.0,4,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,99.0,0.07,...,NaN,99.0,2.0,1.0,NaN,2.0,-99,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03:00,1,2,99,2,2,1,1,2
6306,058-0001-0118,1,44,751.812637,1,40,1,2,8,1,NaN,NaN,1,1,1,3,8501,92311,9,1,NaN,1,8.0,13.0,5.0,1.0,4.0,2.0,1.0,1.0,NaN,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,1.0,NaN,35000,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,04:00,3,4,9,6,2,1,1,1
3977,022-0080-0034,1,41,3651.991043,1,35,1,2,4,1,NaN,NaN,1,1,1,3,4803,34323,4,2,1.0,2,4.0,4.0,4.0,2.0,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,10000,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,01:00,3,3,8,2,2,1,2,1
8189,082-0078-0008,1,43,10857.331376,1,63,1,2,2,1,NaN,NaN,1,1,1,3,8101,56313,7,2,6.0,1,1.0,21.0,21.0,1.0,4.0,3.0,1.0,1.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,1.0,NaN,15000,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02:00,4,5,6,4,3,1,1,1


La columna C3P16.1 pertenece al Sexo (1 correspondiente a varon y 2 correspondiente a mujer). Aunque la consigna hace referencia al género y no al sexo, como en la encuesta no hay referencia al genero, el analisis se hara sobre el sexo de las personas encuestadas.



Por lo pronto no se manejara ninguna alteración o eliminación de los features, ya que la idea es que los algoritmos se encarguen de eso. Aún asi dropeamos el codigo de muestra por lo que es solo una enumeración sin relevancía para nosotros. Ademas de remplazar los valores atipicos y los NaN por cero. (lo de los valores atipicos puede ser muy riesgoso pero se remplazan por cero por que la idea es normalizar los atributos)

In [ ]:
df = df.drop(['codmuestra'],1)

df.fillna(0, inplace=True)
df = df.replace(-99,0)
df = df.replace(99,0)
df['C3P15.1'] = df['C3P15.1'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[-1]))  #Paso la duración a minutos

df.sample(10)

,hogar,region,ponderaocup,C0P10.2,C0P10.3,C3P16.1,C3P16.2,nivel_ed,C3P16.6,C3P16.6.1cod,C3P16.7,C3P16.8,C1P2.1,C1P2.6,cat_ocup,rama_caes,ocupa_cno,ramacaes_o,sector,establecimiento,sit_oc,tamanio,antigestab_a,antigpuesto,registracion,upreg,tickets,quien_contrato,C1P3.1.10,C1P3.1.11,C1P3.1.12,C1P3.1.15,C1P3.1.16,C1P3.2.10,C1P3.2.11,C1P3.2.12,C1P3.3.1,C1P3.3.2,antigestab_sd,C1P3.3.4,...,C2P12.15,C1P3.3.14,C1P3.3.19,aumentosal,C2P12.11,C1P3.3.18,ingreso_op,C2P13.1.1,C2P13.1.2,C2P13.1.3,C2P13.1.4,C2P13.1.5,C2P13.1.6,C2P13.1.7,C2P13.1.8,C2P13.1.9,C2P13.1.10,C2P13.4.2,C2P13.4.3,C2P13.4.4,C2P13.4.5,C2BP13.2,C2BP13.3,C3P14.2,C3P14.3,C3P14.4,C3P14.5,C3P14.6,C3P14.7,C3P14.8,trabdom_noremu,C3P15.1,C3P15.2,C3P15.3,C3P17.1,edadhogar,pershog_ag,edad_0a5,edad_6a11,edad_12a17
8772,1,44,350.208807,3,20,2,1,4,2,202.0,2005.0,3,2,2,3,9700,57314,11,2,5.0,4,0.0,0.0,0.00,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,0.01,2.0,...,0.0,2.0,2.0,2.0,0.0,0.0,0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1,60,5,5,0,4,3,1,1,1
8799,1,44,146.973472,1,37,1,2,1,2,202.0,2008.0,3,1,1,3,4000,72313,3,2,1.0,1,4.0,10.0,10.00,1.0,4.0,0.0,1.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,50000,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,180,5,5,9,2,3,1,2,1
2789,1,43,2382.043018,2,43,2,2,4,1,0.0,0.0,1,1,1,3,8501,56314,9,1,0.0,1,5.0,21.0,21.00,1.0,1.0,3.0,1.0,1.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,12000,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,720,1,3,6,2,3,1,2,2
4703,1,41,534.146729,1,39,2,1,3,1,0.0,0.0,3,1,1,2,9602,40112,12,2,5.0,8,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,300,1,5,2,2,3,2,1,2
6295,1,41,562.815564,1,32,1,2,3,1,0.0,0.0,1,1,2,3,101,34323,1,2,1.0,2,4.0,2.0,2.00,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,1.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,10000,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,180,5,2,3,2,3,2,2,1
8912,1,44,248.846544,1,25,1,2,3,1,0.0,0.0,1,1,1,3,2701,56314,2,2,1.0,1,6.0,5.0,5.00,1.0,4.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,15000,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,30,4,5,5,2,2,2,1,1
8368,1,40,413.583372,1,40,2,1,2,1,0.0,0.0,1,1,1,3,9700,55314,11,2,5.0,4,0.0,0.0,0.00,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.07,1.0,...,0.0,2.0,2.0,2.0,0.0,0.0,6000,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,180,3,3,2,2,3,2,1,1
7248,1,42,1172.791915,3,22,1,1,3,1,0.0,0.0,1,1,2,3,101,60313,1,2,1.0,2,5.0,3.0,3.00,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,1.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,10000,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0,2,2,4,2,4,1,2,2
5143,1,43,455.840761,1,38,1,2,2,1,0.0,0.0,1,1,1,3,8401,56314,8,1,0.0,1,8.0,5.0,2.00,1.0,0.0,1.0,1.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0,3,3,0,6,2,1,1,1
402,1,10,483.377108,1,32,2,1,7,1,0.0,0.0,1,1,1,3,6200,10332,6,2,2.0,1,9.0,7.0,0.07,1.0,4.0,1.0,1.0,1.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,1.0,3.0,0.0,28000,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,120,1,1,6,6,1,1,1,1


---
Para la comparación de distintos metodos de reducción de la dimensionalidad, usamos un clasificador para ver los resultados de prescición y como varian con los atributos resultantes de la reducción.
Para esto se eligio el clasificador de RandomForest pensando que seria el mas adecuado.

Entonces primero aplicamos un metodo lineal bastante utilizado que es el PCA, sepranado ya nuestro dataset para el futuro entrenamiento de nuestro clasificador.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler


s = StandardScaler()


x_train,  x_test, y_train,  y_test  = train_test_split(df.drop(['C3P16.1'],1), df['C3P16.1'], test_size = 0.2, random_state = 0)

x_train = s.fit_transform(x_train)
x_test = s.transform(x_test)

A priori no se descarta ningún feature, para ver como varian los pesos de estos segun el algoritmo.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

pca = PCA()

x_train = pca.fit_transform(x_train)
x_test = pca.transform(x_test)

explained_variance = pca.explained_variance_ratio_
print("Varianza : ", explained_variance)

Varianza :  [1.87710966e-01 5.51231649e-02 4.53709073e-02 3.07480972e-02
 2.88058250e-02 2.49246076e-02 2.10077105e-02 2.03866547e-02
 1.65874507e-02 1.51631089e-02 1.48770226e-02 1.40865222e-02
 1.36406747e-02 1.26021716e-02 1.23209327e-02 1.21078939e-02
 1.09546691e-02 1.07486808e-02 1.02454797e-02 9.77054591e-03
 9.51845504e-03 9.07229215e-03 7.68746332e-03 6.78704491e-03
 6.36738570e-03 6.03151659e-03 5.56103118e-03 5.26433746e-03
 4.97242575e-03 4.88843570e-03 4.77844247e-03 4.65012128e-03
 4.53439732e-03 4.39577266e-03 4.32895663e-03 4.23489281e-03
 4.05694353e-03 3.97809156e-03 3.94365368e-03 3.88342261e-03
 3.84955511e-03 3.69848158e-03 3.67059156e-03 3.56213615e-03
 3.49429171e-03 3.47994105e-03 3.41788335e-03 3.38342461e-03
 3.22954538e-03 3.15945002e-03 3.15353360e-03 3.05797058e-03
 3.02315723e-03 2.98231406e-03 2.95778076e-03 2.88977297e-03
 2.86426201e-03 2.82259200e-03 2.80205566e-03 2.77563121e-03
 2.75762877e-03 2.71612806e-03 2.70129077e-03 2.67479708e-03
 2.66513325e

Vemos que ya los principales features no poseen un gran peso para la estimación, por lo que vamos a tener que utilizar varios si queremos que nuestro clasificador tenga una utilidad moderada, para la primera dejemos al menos 10.

In [ ]:
pca = PCA(n_components= 10, svd_solver='auto')

x_train = pca.fit_transform(x_train)
x_test = pca.transform(x_test)

explained_variance = pca.explained_variance_ratio_
print("Varianza : ", explained_variance)

Varianza :  [0.33095411 0.09718792 0.07999367 0.05421212 0.05078769 0.0439447
 0.0370388  0.03594381 0.02924541 0.02673415]


In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(max_depth=2, random_state=0)
classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('\n Accuracy :' , accuracy_score(y_test, y_pred))

[[1016    7]
 [ 619  152]]

 Accuracy : 0.6510590858416946


Aumentamos hasta 20 features para comprobar si se justifica la reducción de parametros


In [ ]:
pca = PCA(n_components= 20, svd_solver='auto')

x_train = pca.fit_transform(x_train)
x_test = pca.transform(x_test)


classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('\n Accuracy :' , accuracy_score(y_test, y_pred))

[[994  29]
 [556 215]]

 Accuracy : 0.6739130434782609


Mejoro, pero usamos el doble de de features para solo un 2% de precisión. Y usando todos?

In [ ]:
pca = PCA(svd_solver='auto')

x_train = pca.fit_transform(x_train)
x_test = pca.transform(x_test)


classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('\n Accuracy :' , accuracy_score(y_test, y_pred))

[[1020    3]
 [ 630  141]]

 Accuracy : 0.6471571906354515


Aca ya vemos que tenemos menos acuraccy usando todos los parametros, por lo que se aprecia la llamada "maldición de la dimensionalidad" y confirmamos que no siempre es bueno usar todas las features. 

--- 

Ahora hacemos la prueba con metodos no lineales, probemos con el metodo t-sne 

In [ ]:
x_train,  x_test, y_train,  y_test  = train_test_split(df.drop(['C3P16.1'],1), df['C3P16.1'], test_size = 0.2, random_state = 0)

x_train = s.fit_transform(x_train)
x_test = s.transform(x_test)

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, learning_rate='auto', init='random', init='pca')

x_train = tsne.fit_transform(x_train)
x_test = tsne.fit_transform(x_test)

print(x_train.shape)
print(x_test.shape)

(7172, 2)
(1794, 2)


In [ ]:
classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('\n Accuracy :' , accuracy_score(y_test, y_pred))

[[873 150]
 [737  34]]

 Accuracy : 0.5055741360089187


In [ ]:
tsne = TSNE(n_components=3, learning_rate='auto', init='pca')

x_train = tsne.fit_transform(x_train)
x_test = tsne.fit_transform(x_test)

print(x_train.shape)
print(x_test.shape)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,


(7172, 3)
(1794, 3)


In [ ]:
classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('\n Accuracy :' , accuracy_score(y_test, y_pred))

[[840 183]
 [594 177]]

 Accuracy : 0.5668896321070234


### Vemos que para el caso de T-sne no es comparable al de PCA porque nuestro maximo numero de features es mucho menor al que adoptamos anteriormente, por lo que si vemos que tiene un prescición menor que el metodo de PCA, esta se debe seguramente a los pocos parametros tomados, por lo que probaremos con otro metodo no lineal que se puedan adoptar más parametros.
----
----

In [ ]:
from sklearn.datasets import load_digits
from sklearn.manifold import MDS

mds = MDS(n_components=10, dissimilarity='euclidean')

x_train = mds.fit_transform(x_train)
x_test = mds.fit_transform(x_test)

In [ ]:
classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('\n Accuracy :' , accuracy_score(y_test, y_pred))

[[859 164]
 [633 138]]

 Accuracy : 0.5557413600891862


In [ ]:

mds = MDS(n_components=20, dissimilarity='euclidean')

x_train = mds.fit_transform(x_train)
x_test = mds.fit_transform(x_test)

In [ ]:
classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('\n Accuracy :' , accuracy_score(y_test, y_pred))

[[939  84]
 [628 143]]

 Accuracy : 0.6031215161649944


In [ ]:
mds = MDS(n_components=40, dissimilarity='euclidean')

x_train = mds.fit_transform(x_train)
x_test = mds.fit_transform(x_test)

In [ ]:
classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('\n Accuracy :' , accuracy_score(y_test, y_pred))

[[918 105]
 [684  87]]

 Accuracy : 0.560200668896321


Vemos que este metodo, por mas que sea no lineal, se corresponde a los resultados antes vistos en el PCA con respecto a la prescición del clasificador con la cantidad de features elegidos. Tambien se ve que las presciciónes son un poco menor que para el caso lineal, esto probablemente ocurre por la naturaleza de los parametros que se encuentran en la encuesta.
Concluimos nuevamente, que una correcta elección o con suficiente tiempo de pruebas, siempre podemos tener mas y mayores resultados en lo que de aprendizaje de maquinas se trata.